# MedGemma 1.5 AutoRound 4-bit + RadGraph F1 评估

**模型**: [Ashley101179/medgemma-1.5-4b-it-4bit-autoround](https://huggingface.co/Ashley101179/medgemma-1.5-4b-it-4bit-autoround)

- **量化方法**: Intel AutoRound (GPTQ format)
- **位数**: 4-bit
- **显存**: ~4GB (原模型 ~10GB)
- **评估**: RadGraph F1 on MIMIC-CXR

## 环境说明
- **Python**: 3.12（Kaggle 默认，无需降级）
- **GPU**: Kaggle P100
- **依赖**: transformers 4.47.1 + auto-gptq

# Cell 1: 安装依赖

In [ ]:
# Kaggle P100: transformers 4.47.1 + auto-gptq
import subprocess, sys, os, shutil

TF_ENV = "/kaggle/working/tf_env"
CONSTRAINTS = "/kaggle/working/constraints.txt"

# 清理旧环境
if os.path.exists(TF_ENV):
    shutil.rmtree(TF_ENV)

# 1. 使用 Kaggle 预装 PyTorch（通常是 2.x + cu121，兼容 P100）
print("使用 Kaggle 预装的 PyTorch...")

# 2. 创建约束文件（强制锁定 transformers 版本）
with open(CONSTRAINTS, "w") as f:
    f.write("transformers==4.47.1\n")

# 3. 安装 transformers 到 TF_ENV（使用清华镜像）
subprocess.run([
    "pip", "install", "--target", TF_ENV, "-q", "-c", CONSTRAINTS,
    "transformers==4.47.1", "pillow<12", "jinja2",
    "-i", "https://pypi.tuna.tsinghua.edu.cn/simple"
], check=True)

# 4. 安装 auto-gptq（GPTQ 后端，使用清华镜像）
print("安装 auto-gptq...")
subprocess.run([
    "pip", "install", "-q", "auto-gptq", "optimum", "accelerate",
    "-i", "https://pypi.tuna.tsinghua.edu.cn/simple"
], check=True)

# 5. 安装 radgraph 到系统（使用清华镜像）
subprocess.run([
    "pip", "install", "-q", "radgraph",
    "-i", "https://pypi.tuna.tsinghua.edu.cn/simple"
], check=True)

# 6. 设置路径优先级
sys.path.insert(0, TF_ENV)

print("✅ 安装完成，请立刻 Restart Session，然后 Run All")

# Cell 2: HuggingFace 登录

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

try:
    tok = UserSecretsClient().get_secret("zhuxirui11")
    if tok:
        login(token=tok)
        print("✅ HF 登录成功")
    else:
        print("⚠️ Token 为空")
except Exception as e:
    print("❌ 未配置 zhuxirui11 Secret")
    print("\n请按以下步骤配置：")
    print("1. 点击右侧 'Add-ons' → 'Secrets'")
    print("2. Label 填写：zhuxirui11")
    print("3. Value 填写你的 HF token（从 https://huggingface.co/settings/tokens 获取）")
    print("4. 保存后重新运行本 Cell")

# Cell 3: 环境检查

In [ ]:
import sys, torch

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    USE_BF16 = torch.cuda.get_device_capability(0)[0] >= 8
    DTYPE = torch.bfloat16 if USE_BF16 else torch.float16
    print(f"Precision: {'BF16' if USE_BF16 else 'FP16 (P100)'}")
else:
    DTYPE = torch.float32
    print("⚠️ No GPU, using FP32")

# Cell 4: 路径配置与版本检查

In [ ]:
import sys, os, gc
import pandas as pd
from PIL import Image
from tqdm import tqdm

# 确保 TF_ENV 优先
TF_ENV = "/kaggle/working/tf_env"
if TF_ENV not in sys.path:
    sys.path.insert(0, TF_ENV)
elif sys.path[0] != TF_ENV:
    sys.path.remove(TF_ENV)
    sys.path.insert(0, TF_ENV)

# Kaggle 数据集路径
DATASET_ROOT = "/kaggle/input/datasets/simhadrisadaram/mimic-cxr-dataset/official_data_iccv_final"
CSV_CANDIDATES = [
    "/kaggle/input/datasets/xiruizhu1111/clean-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/datasets/xiruizhu1111/clean-train-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/clean-data/mimic_eval_single_image_final_233.csv",
    "/kaggle/working/mimic_eval_single_image_final_233.csv",
]

# 查找 CSV 文件
CSV_PATH = None
for p in CSV_CANDIDATES:
    if os.path.exists(p):
        CSV_PATH = p
        break

if not CSV_PATH:
    print("❌ 未找到 CSV 文件！")
    print("尝试的路径：")
    for p in CSV_CANDIDATES:
        print(f"  - {p}")
    print("\n请检查：")
    print("1. 是否已添加包含 CSV 的数据集？（右侧 Add Data）")
    print("2. CSV 文件名是否为 'mimic_eval_single_image_final_233.csv'？")
    raise FileNotFoundError("CSV 文件不存在")

# 验证 transformers 版本
import transformers
tf_ver = transformers.__version__
if tf_ver.startswith('5.'):
    raise RuntimeError(
        f"❌ transformers {tf_ver} 不兼容！需要 4.47.1\n"
        f"解决方法：重新运行 Cell 1，确保看到 'transformers-4.47.1'，然后 Restart Session"
    )
print(f"✅ transformers: {tf_ver}")

print(f"Dataset: {DATASET_ROOT}")
print(f"✅ CSV: {CSV_PATH}")

# Cell 4.5: CSV 检查

In [ ]:
# 检查 CSV 文件结构
import pandas as pd

df_check = pd.read_csv(CSV_PATH)

print(f"✅ CSV 文件加载成功！")
print(f"总行数: {len(df_check)}")
print(f"\n列名: {list(df_check.columns)}")
print(f"\n前 3 行数据预览：")
print(df_check.head(3))

# 检查关键列是否存在
print("\n=== 关键列检查 ===")
has_img_path = "Image_Path" in df_check.columns
has_ground_truth = "Ground_Truth" in df_check.columns or "text" in df_check.columns
has_subject_id = "subject_id" in df_check.columns
has_view = "View" in df_check.columns

print(f"Image_Path 列: {'✅ 存在' if has_img_path else '❌ 不存在（将查找 PA/AP/Lateral 列）'}")
print(f"Ground_Truth/text 列: {'✅ 存在' if has_ground_truth else '❌ 不存在'}")
print(f"subject_id 列: {'✅ 存在' if has_subject_id else '⚠️ 不存在'}")
print(f"View 列: {'✅ 存在' if has_view else '⚠️ 不存在（将默认使用 PA）'}")

if not has_img_path:
    view_cols = [c for c in ['PA', 'AP', 'Lateral'] if c in df_check.columns]
    if view_cols:
        print(f"\n找到视图列: {view_cols}")
        print(f"示例路径: {df_check[view_cols[0]].iloc[0]}")
    else:
        print("\n❌ 警告：未找到图片路径列（Image_Path 或 PA/AP/Lateral）")

# Cell 5: 加载 AutoRound 4-bit 量化模型

In [ ]:
# 清除 transformers 缓存
for k in list(sys.modules.keys()):
    if k == "transformers" or k.startswith("transformers."):
        del sys.modules[k]

from transformers import AutoProcessor, AutoModelForImageTextToText

model_id = "Ashley101179/medgemma-1.5-4b-it-4bit-autoround"
print(f"Loading AutoRound 4-bit model ({DTYPE})...")
print("⚠️ 首次加载会下载 ~2.5GB 权重，请耐心等待...\n")

# 加载 GPTQ 量化模型（AutoRound 格式）
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

if torch.cuda.is_available():
    mem_gb = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"✅ AutoRound 4-bit model loaded")
    print(f"GPU memory: {mem_gb:.2f} GB (预期 ~4GB)")
    torch.cuda.reset_peak_memory_stats()

# Cell 6: 生成报告

In [ ]:
import re

PROMPT_TEMPLATE = (
    "You are an expert radiologist. Describe this {view} view chest X-ray. "
    "Provide a concise report consisting of Findings and Impression. "
    "Focus on the heart, lungs, mediastinum, pleural space, and bones. "
    "Do NOT use bullet points, asterisks, or section headers. "
    "Do NOT include disclaimers or 'AI' warnings. Output pure medical text only."
)

def fix_image_path(path):
    """修正 CSV 中的图片路径为实际 Kaggle datasets 路径"""
    if pd.isna(path) or not path:
        return None
    path = str(path).strip()
    
    if path.startswith("/kaggle/input/datasets/"):
        return path if os.path.exists(path) else None
    
    if path.startswith("/kaggle/input/mimic-cxr-dataset/"):
        new_path = path.replace("/kaggle/input/mimic-cxr-dataset/", "/kaggle/input/datasets/simhadrisadaram/mimic-cxr-dataset/")
        return new_path if os.path.exists(new_path) else None
    
    if not path.startswith("/"):
        full = os.path.join(DATASET_ROOT, path)
        return full if os.path.exists(full) else None
    
    return path if os.path.exists(path) else None

def get_single_image_path(cell_val):
    if pd.isna(cell_val):
        return None
    s = str(cell_val).strip().replace("[", "").replace("]", "").replace("'", "").replace('"', "").split(",")[0].strip()
    if "files" in s:
        rel = "files" + s.split("files", 1)[1]
    else:
        rel = s.strip("/")
    full = os.path.join(DATASET_ROOT, rel) if not rel.startswith("/") else rel
    return fix_image_path(full)

def generate_report(model, processor, img_path, view="PA"):
    if not os.path.exists(img_path):
        return ""
    try:
        img = Image.open(img_path).convert("RGB")
    except:
        return ""
    prompt = PROMPT_TEMPLATE.format(view=view)
    msgs = [{"role": "user", "content": [{"type": "image", "image": img}, {"type": "text", "text": prompt}]}]
    inp = processor.apply_chat_template(msgs, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt").to(model.device, dtype=DTYPE)
    L = inp["input_ids"].shape[-1]
    with torch.inference_mode():
        out = model.generate(
            **inp,
            max_length=None,  # 禁用默认 max_length
            max_new_tokens=300,  # 使用相对长度
            min_new_tokens=5,  # 强制最少生成
            pad_token_id=0,  # 显式设置 pad token
            do_sample=False
        )
    txt = processor.decode(out[0][L:], skip_special_tokens=True)
    return re.sub(r'\s+', ' ', txt.replace("Findings:", "").replace("Impression:", "")).strip()

# 生成报告
df = pd.read_csv(CSV_PATH)
GT_COL = "Ground_Truth" if "Ground_Truth" in df.columns else "text"
IMG_COL = "Image_Path" if "Image_Path" in df.columns else None

rows_out = []
NUM = min(50, len(df))

for idx, row in tqdm(df.head(NUM).iterrows(), total=NUM, desc="Generating AutoRound 4-bit reports"):
    path, view = None, "PA"
    if IMG_COL:
        path = fix_image_path(row.get(IMG_COL))
        view = row.get("View", "PA")
    else:
        for c, v in [("PA", "PA"), ("AP", "AP"), ("Lateral", "Lateral")]:
            if c in df.columns and (p := get_single_image_path(row.get(c))):
                path, view = p, v
                break
    if not path:
        continue
    gt = str(row.get(GT_COL) or "").strip()
    if not gt or gt.startswith("You are"):
        continue
    rep = generate_report(model, processor, path, view)
    rows_out.append({
        "subject_id": row["subject_id"],
        "View": view,
        "Image_Path": path,
        "Ground_Truth": gt,
        "Generated_Report": rep
    })

df_sub = pd.DataFrame(rows_out)
print(f"\n✅ Generated {len(df_sub)} reports")

# 快速检查生成质量
print("\n=== 生成报告检查 ===")
print(f"总共生成: {len(df_sub)} 条报告")
print(f"前 3 条数据：")
for i in range(min(3, len(df_sub))):
    print(f"\n[{i}] Ground Truth: {df_sub.iloc[i]['Ground_Truth'][:80]}...")
    print(f"    Generated: {df_sub.iloc[i]['Generated_Report'][:80]}...")

empty_count = (df_sub['Generated_Report'].str.strip() == "").sum()
print(f"\n空报告数量: {empty_count} / {len(df_sub)}")
if empty_count > len(df_sub) * 0.5:
    print("❌ 警告：超过一半的报告为空！")

# Cell 7a: RadGraph 兼容性修复（必须先运行！）

**说明**：修复 `transformers` 5.x 与 `radgraph` 的兼容性问题

In [ ]:
from transformers import BertTokenizer

fixed_methods = []

# 1. 添加 encode_plus 方法
if not hasattr(BertTokenizer, 'encode_plus'):
    def encode_plus_wrapper(self, text, *args, **kwargs):
        return self(text, *args, **kwargs)
    BertTokenizer.encode_plus = encode_plus_wrapper
    fixed_methods.append('encode_plus')

# 2. 添加 build_inputs_with_special_tokens 方法
if not hasattr(BertTokenizer, 'build_inputs_with_special_tokens'):
    def build_inputs_with_special_tokens_wrapper(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        return [self.cls_token_id] + token_ids_0 + [self.sep_token_id] + token_ids_1 + [self.sep_token_id]
    BertTokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens_wrapper
    fixed_methods.append('build_inputs_with_special_tokens')

# 3. 添加 get_special_tokens_mask 方法
if not hasattr(BertTokenizer, 'get_special_tokens_mask'):
    def get_special_tokens_mask_wrapper(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        if already_has_special_tokens:
            return super(BertTokenizer, self).get_special_tokens_mask(
                token_ids_0=token_ids_0, token_ids_1=token_ids_1, already_has_special_tokens=True
            )
        if token_ids_1 is None:
            return [1] + ([0] * len(token_ids_0)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
    BertTokenizer.get_special_tokens_mask = get_special_tokens_mask_wrapper
    fixed_methods.append('get_special_tokens_mask')

if fixed_methods:
    print(f"✅ RadGraph 兼容性修复已应用: {', '.join(fixed_methods)}")
else:
    print("✅ BertTokenizer 已有所有必需方法")

# Cell 7b: RadGraph F1 评估

**指标说明**（百分制）：
- **RG_E**: Entity F1（实体匹配）
- **RG_ER**: Entity + Relation F1（实体+关系，**论文常用指标**）
- **RG_ER_bar**: Complete Match F1（完全匹配）

In [ ]:
from radgraph import F1RadGraph
import torch

print("🔍 开始 RadGraph F1 评估...")
print(f"评估样本数: {len(df_sub)}\n")

# 准备数据
refs = df_sub["Ground_Truth"].fillna("").tolist()
hyps = df_sub["Generated_Report"].fillna("").tolist()

# 过滤掉空报告
valid_pairs = [(h, r) for h, r in zip(hyps, refs) if h and r and len(h.strip()) > 0]
if len(valid_pairs) < len(hyps):
    print(f"⚠️ 跳过 {len(hyps) - len(valid_pairs)} 条空报告\n")
hyps_clean, refs_clean = zip(*valid_pairs) if valid_pairs else ([], [])

# 初始化 RadGraph
print("Using device:", "cuda:0" if torch.cuda.is_available() else "cpu")
f1radgraph = F1RadGraph(reward_level="all")

# 计算 F1
results = f1radgraph(hyps=list(hyps_clean), refs=list(refs_clean))

# 解析返回值：results[0] = (simple, partial, complete)
avg_scores = results[0]
simple_f1 = float(avg_scores[0])    # RG_ER_bar
partial_f1 = float(avg_scores[1])   # RG_ER (论文常用)
complete_f1 = float(avg_scores[2])  # RG_E

# 显示结果（百分制）
print("\n" + "=" * 60)
print("AutoRound 4-bit RadGraph F1 评估结果（百分制）")
print("-" * 60)
print(f"RG_E (Entity):              {complete_f1*100:.2f}")
print(f"RG_ER (Entity+Relation):    {partial_f1*100:.2f}  ← 论文常用")
print(f"RG_ER_bar (Complete Match): {simple_f1*100:.2f}")
print("=" * 60)

if torch.cuda.is_available():
    peak_mem = torch.cuda.max_memory_allocated(0) / 1e9
    print(f"\nPeak GPU 显存: {peak_mem:.2f} GB")

# 保存结果
df_sub.to_csv("/kaggle/working/medgemma_autoround_4bit_reports.csv", index=False)
print("\n✅ 结果已保存至 /kaggle/working/medgemma_autoround_4bit_reports.csv")
print("\n✅ 评估完成！")